## Extracting LSTM features and Feature Maps from the main data
To better understand the code, check out `./prep_files/preprocessing_synced_for_MTL.ipynb`. That notebook explains the thought process behind this code on a sample dataset whilst this goes straight to the point.

In [2]:
import pandas as pd
import numpy as np
import src.preprocess
from src.preprocess import LSTM_featureExtractor, Prep_data_for_CNN

In [2]:
data = pd.read_csv('./data/concat_users.csv')

In [3]:
data.groupby(['User', 'Position'])['Segment'].count()

User   Position
User1  Bag         304434
       Hand        304438
       Hips        304434
       Torso       304430
User2  Bag         372171
       Hand        372170
       Hips        372172
       Torso       372177
User3  Bag         347627
       Hand        347627
       Hips        347627
       Torso       347626
Name: Segment, dtype: int64

In [7]:
sub_segmented_data = src.preprocess.create_sub_segments(data, window_size=112, overlap=0.5)
sub_segmented_data.to_csv('./data/sub_segmented_data.csv', index=False, header=True)

In [10]:
del data, sub_segmented_data

-----

In [12]:
subbed_data = pd.read_csv('./data/sub_segmented_data.csv')

In [13]:
subbed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8092672 entries, 0 to 8092671
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   time_ms         float64
 1   acceleration_x  float64
 2   acceleration_y  float64
 3   acceleration_z  float64
 4   gyroscope_x     float64
 5   gyroscope_y     float64
 6   gyroscope_z     float64
 7   orientation_w   float64
 8   orientation_x   float64
 9   orientation_y   float64
 10  orientation_z   float64
 11  Date            int64  
 12  Position        object 
 13  Coarse_label    float64
 14  Fine_label      float64
 15  Segment         int64  
 16  User            object 
 17  Sub_Segment     int64  
dtypes: float64(13), int64(3), object(2)
memory usage: 1.1+ GB


### Feature Extraction for LSTM

In [14]:
lstm_featExtract = LSTM_featureExtractor(subbed_data, num_features=6, window=112)

In [15]:
lstm_features, lstm_labels = lstm_featExtract.feature_extractor(start=0)

In [18]:
np.save('./data/lstm_features_labels/full_lstm_features.npy', lstm_features)
np.save('./data/lstm_features_labels/full_lstm_labels.npy', lstm_labels)

In [19]:
print(lstm_features.shape)  # Should be (num_samples, window, num_features)
print(lstm_labels.shape)    # Should be (num_samples,)

(72256, 112, 6)
(72256,)


In [20]:
del lstm_features, lstm_labels

### Feature Extraction for CNN-RNN

In [21]:
subbed_data.query("Coarse_label == 5.0 & Position=='Hand'").groupby(['User','Segment'])[['Sub_Segment']].count()

Sub_Segment
User  Segment             
User1 5              35728
      7              33040
      9              18928
      12             11088
User2 64             67088
      66             14672
      70             26880
      73             27440
      75             17584
User3 22             50288

This data has 3 drivers whilst my sample had 2.

In [22]:
subbed_data.query("Coarse_label == 5.0").groupby('User')[['Segment','Sub_Segment']].nunique()

,Segment,Sub_Segment
User,,
User1,4,882
User2,5,1372
User3,1,449


In [29]:
windowed_df = Prep_data_for_CNN().get_window_df_from_dir('./data/sub_segmented_data.csv',
                                                         prep_for_FMAPextract=False)

time it took: `304m 0.5s`

In [30]:
windowed_df.to_csv('./data/windowed_df.csv', index=False, header=True)

I should have set `prep_for_FMAPextract=True` because I need the full windowed_df to be able to split it into train and test without leaking data into the SimpleCNN. I'll do it manually.

In [31]:
len(windowed_df)

36304

In [33]:
windowed_df['user'] = windowed_df['user_act'].values
del windowed_df['user_act']

In [36]:
windowed_df = Prep_data_for_CNN().prep_for_FMAPextract(subbed_data, windowed_df)

# change other labels to `Not_driving`  (a dummy label)
windowed_df['user_act'] = windowed_df['user'].values
idx_users = windowed_df.query('coarse_label == 5.0').index
df_with_drivers = windowed_df.index.isin(idx_users)
windowed_df.loc[~df_with_drivers, 'user'] = 'Not_driving'

In [37]:
windowed_df.to_csv('./data/full_windowed_df.csv', index=False, header=True)

In [38]:
len(windowed_df)

72256

This `full_windowed_df` is made up of duplicates and interpolated data. As a result, I'll need to take 80:20 of the duplicates (original and corr dupe) as well as 80:20 of interpolated data.

In [27]:
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')

windowed_df = pd.read_csv(
    './data/full_windowed_df.csv',
    converters={
        'long_acc_window': converter,
        'tranv_acc_window': converter,
        'ang_vel_window': converter
    }
)

In [28]:
windowed_df.columns

Index(['user', 'coarse_label', 'segment', 'position', 'window_index',
       'long_acc_window', 'tranv_acc_window', 'ang_vel_window', 'Interp_Flag',
       'Dupl_Flag', 'Train_Flag', 'user_act'],
      dtype='object')

In [29]:
idx_non_dupe = windowed_df.query("Dupl_Flag==0 & Interp_Flag==0").index
idx_interp = windowed_df.query("Dupl_Flag==0 & Interp_Flag==1").index

For each non_dupe, it's corresponding duplicate is the next idx

Get training and test index

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
tr_idx_nondupe, test_idx_nondupe = train_test_split(idx_non_dupe,
                                                    test_size=0.2,
                                                    stratify=windowed_df.loc[idx_non_dupe, 'user_act'].values,
                                                    random_state=42)

tr_idx_interp, test_idx_interp = train_test_split(idx_interp,
                                                  test_size=0.2,
                                                  stratify=windowed_df.loc[idx_interp, 'user_act'].values,
                                                  random_state=42)

In [33]:
# get the idx for the duplicates
tr_idx_dupe = tr_idx_nondupe+1
test_idx_dupe = test_idx_nondupe+1

In [34]:
train_idx = np.concatenate([tr_idx_nondupe, tr_idx_dupe, tr_idx_interp], axis=None)
test_idx = np.concatenate([test_idx_nondupe, test_idx_dupe, test_idx_interp], axis=None)

In [35]:
## subsetting only the training data
windowed_df = windowed_df.loc[train_idx, :]

Begin hyperparam tuning for SimpleCNN

In [36]:
X, y, uniq = Prep_data_for_CNN().prep_input_for_CNN(windowed_df)

In [37]:
uniq, np.unique(y)

(array(['Not_driving', 'User1', 'User2', 'User3'], dtype='<U11'),
 array([0, 1, 2, 3], dtype=int64))

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [40]:
X.shape

(57803, 3, 224)

In [41]:
# Perform a train-test split
np.random.seed(42)
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_tr, y_tr, test_size=0.2, stratify=y_tr)

class_weights = torch.tensor(compute_class_weight('balanced', classes=np.unique(y_train), y=y_train), dtype=torch.float32)
class_weights = class_weights.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# normalizing X
mean = X_train.mean(axis=(0, 2), keepdims=True)
std = X_train.std(axis=(0, 2), keepdims=True)

X_train = (X_train - mean) / std
X_valid = (X_valid - mean) / std
X_test = (X_test - mean) / std

In [42]:
from src.dataset import SignalsDataset

In [43]:
# Create datasets for each feature
feature_indices = {'long_acc': 0, 'tranv_acc': 1, 'ang_vel': 2}

train_datasets = {feature: SignalsDataset(X_train, y_train, idx) for feature, idx in feature_indices.items()}
valid_datasets = {feature: SignalsDataset(X_valid, y_valid, idx) for feature, idx in feature_indices.items()}
test_datasets = {feature: SignalsDataset(X_test, y_test, idx) for feature, idx in feature_indices.items()}

In [44]:
from src.engine import simpleCNN_engine
from src.model_simpleCNN import SimpleCNN
from src.dataset import SignalsDataset
from src.preprocess import Prep_data_for_CNN
from src.hyperparam import RayTuning
from ray import tune, train

from IPython.display import display, HTML

# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">')  # Ensure the path is correct
display(css)

In [46]:
SimpleCNN()

SimpleCNN(
  (conv1): Conv1d(1, 224, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=50176, out_features=4, bias=True)
)

Hyperparam Tuning - Long Model

In [47]:
# Enhanced Hyperparameter search space
config = {
    "optimizer": tune.choice(["adam"]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "scheduler": tune.choice(["exp"]),
    "gamma": tune.uniform(0.1, 0.9),
    "epochs": tune.choice([30]),
    "batch_size": tune.choice([32])
}

criterion = nn.CrossEntropyLoss(weight=class_weights)
save_dir = 'SimpleCNN_long'
model = SimpleCNN
modelType = 'SimpleCNN'
engine = simpleCNN_engine

In [48]:
mytuner = RayTuning(config, save_dir, criterion, model, modelType, engine)

In [49]:
mytuner.main(train_datasets['long_acc'], valid_datasets['long_acc'], 
             test_datasets['long_acc'], num_samples=25)

(train_model pid=21856) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_long/trial_5d70b_00000/checkpoint_000000)
(train_model pid=21856) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_long/trial_5d70b_00000/checkpoint_000001)
(train_model pid=21856) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_long/trial_5d70b_00000/checkpoint_000002)
(train_model pid=21856) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Re

Best trial config: {'optimizer': 'adam', 'lr': 0.00011113828101205411, 'scheduler': 'exp', 'gamma': 0.8492356384312725, 'epochs': 30, 'batch_size': 32}
Best trial final validation loss: 1.1141672693449876
Best trial final validation accuracy: 61.4445%
Test Accuracy: 0.6260704091341579
Best trial test set accuracy: 0.6261%


Hyperparam Tuning - Tranv Model

In [50]:
# Enhanced Hyperparameter search space
config = {
    "optimizer": tune.choice(["adam"]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "scheduler": tune.choice(["exp"]),
    "gamma": tune.uniform(0.1, 0.9),
    "epochs": tune.choice([30]),
    "batch_size": tune.choice([32])
}

criterion = nn.CrossEntropyLoss(weight=class_weights)
save_dir = 'SimpleCNN_tranv'
model = SimpleCNN
modelType = 'SimpleCNN'
engine = simpleCNN_engine

In [51]:
mytuner = RayTuning(config, save_dir, criterion, model, modelType, engine)

In [52]:
mytuner.main(train_datasets['tranv_acc'], valid_datasets['tranv_acc'], 
             test_datasets['tranv_acc'], num_samples=25)

(train_model pid=8396) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_tranv/trial_99950_00000/checkpoint_000000)
(train_model pid=8396) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_tranv/trial_99950_00000/checkpoint_000001)
(train_model pid=8396) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_tranv/trial_99950_00000/checkpoint_000002)
(train_model pid=8396) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Res

2024-07-23 05:18:16,160	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_tranv' in 0.0400s.
2024-07-23 05:18:16,180	INFO tune.py:1041 -- Total run time: 5576.65 seconds (5576.56 seconds for the tuning loop).


Best trial config: {'optimizer': 'adam', 'lr': 0.00015558427999575448, 'scheduler': 'exp', 'gamma': 0.6560277884599833, 'epochs': 30, 'batch_size': 32}
Best trial final validation loss: 1.1554321087639905
Best trial final validation accuracy: 68.9804%
Test Accuracy: 0.6977770089092639
Best trial test set accuracy: 0.6978%


Hyperparam Tuning - Ang. Vel Model

In [53]:
# Enhanced Hyperparameter search space
config = {
    "optimizer": tune.choice(["adam"]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "scheduler": tune.choice(["exp"]),
    "gamma": tune.uniform(0.1, 0.9),
    "epochs": tune.choice([30]),
    "batch_size": tune.choice([32])
}

criterion = nn.CrossEntropyLoss(weight=class_weights)
save_dir = 'SimpleCNN_ang_vel'
model = SimpleCNN
modelType = 'SimpleCNN'
engine = simpleCNN_engine

In [54]:
mytuner = RayTuning(config, save_dir, criterion, model, modelType, engine)

In [55]:
mytuner.main(train_datasets['ang_vel'], valid_datasets['ang_vel'], 
             test_datasets['ang_vel'], num_samples=25)

2024-07-23 06:23:21,580	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_ang_vel' in 0.0300s.
2024-07-23 06:23:21,607	INFO tune.py:1041 -- Total run time: 3889.56 seconds (3889.50 seconds for the tuning loop).


Best trial config: {'optimizer': 'adam', 'lr': 0.00013662142756319453, 'scheduler': 'exp', 'gamma': 0.4740572036017916, 'epochs': 30, 'batch_size': 32}
Best trial final validation loss: 1.138420147525853
Best trial final validation accuracy: 66.7207%
Test Accuracy: 0.6771905544503071
Best trial test set accuracy: 0.6772%


To view tensorboard, run below in terminal;

`tensorboard --logdir="c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/SimpleCNN_long"`

-----

Time to train the model with the given optimal hyperparameters